In [ ]:
from typing import Annotated
from ollama import chat
from typing_extensions import TypedDict
from langchain.schema import HumanMessage, AIMessage  # 导入消息类型

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

In [ ]:
from flask_cors import CORS

# 在 Flask 应用中启用 CORS
CORS(app)

In [ ]:
def chatbot(state: State):
    messages = state["messages"]

    # 格式化消息
    formatted_messages = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            formatted_messages.append(HumanMessage(content=msg.content, role="user"))
        elif isinstance(msg, AIMessage):
            formatted_messages.append(AIMessage(content=msg.content, role="assistant"))
        else:
            formatted_messages.append(HumanMessage(content=msg["content"], role="user"))

    response = ""
    for part in chat('deepseek-r1:1.5b', messages=formatted_messages, stream=True):
        response += part['message']['content']

    return {"messages": [{"role": "assistant", "content": response}]}

In [ ]:
# 添加节点到状态图
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

In [ ]:
@app.route('/')
def web_chat():
    return render_template('index.html')  # 载入聊天页面

In [ ]:
@app.route('/chat', methods=['POST'])
def chat_endpoint():
    print("Received request at /chat endpoint.")  # 添加这行日志
    user_input = request.json.get("user_input")
    if not user_input:
        print("Error: No user input received.")
        return jsonify({"error": "No user input received."}), 400
    print(f"User input: {user_input}")

    state = {"messages": [{"role": "user", "content": user_input}]}
    result = list(graph.stream(state))

    response = result[0]["chatbot"]["messages"][-1]["content"]

    print(f"Assistant response: {response}")

    return jsonify({"assistant_response": response})


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
